In [8]:
%pip install ultralytics opencv-python

In [9]:
%pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [10]:
from ultralytics import YOLO
import gdown
import cv2
import numpy as np
from collections import defaultdict

In [11]:
file_id = "18FyNqEWelu83JAZ5_8TrNlzxQQb7KUjF"
gdown.download(f"https://drive.google.com/uc?id={file_id}", output="video_input.mp4", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=18FyNqEWelu83JAZ5_8TrNlzxQQb7KUjF
To: d:\Projects\Car Detection With YOLO\video_input.mp4
100%|██████████| 1.97M/1.97M [00:06<00:00, 285kB/s]


'video_input.mp4'

In [12]:
model = YOLO("yolo11n.pt") 
video_path = "video_input.mp4" 

cap = cv2.VideoCapture(video_path) 

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_tracking.mp4', fourcc, fps, (width, height))


In [13]:
track_history = defaultdict(lambda: [])
car_ids = set() 

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    results = model.track(frame, persist=True, classes=[2])  
    result = results[0]

    if result.boxes and result.boxes.is_track:
        boxes = result.boxes.xywh.cpu().numpy()
        ids = result.boxes.id.int().cpu().tolist()

        frame = result.plot()

        for box, track_id in zip(boxes, ids):
            x, y, w, h = box
            car_ids.add(track_id)

            track = track_history[track_id]
            track.append((float(x), float(y)))
            if len(track) > 30:
                track.pop(0)

            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(frame, [points], isClosed=False, color=(0, 255, 255), thickness=2)

        cv2.putText(frame, f"Total Cars Detected: {len(car_ids)}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    out.write(frame)


0: 384x640 1 car, 291.1ms
Speed: 18.1ms preprocess, 291.1ms inference, 22.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 191.2ms
Speed: 9.9ms preprocess, 191.2ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 157.3ms
Speed: 3.9ms preprocess, 157.3ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 119.4ms
Speed: 3.3ms preprocess, 119.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 310.9ms
Speed: 3.7ms preprocess, 310.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 145.0ms
Speed: 11.6ms preprocess, 145.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 125.4ms
Speed: 5.2ms preprocess, 125.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 212.7ms
Speed: 21.9ms preprocess, 212.7ms inference, 2.2ms postprocess per image at shape (1, 3, 3

In [17]:
cap.release()
out.release()
print("Video tracking selesai, lihat hasilnya di file output_tracking.mp4")


Video tracking selesai, lihat hasilnya di file output_tracking.mp4
